# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [2]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    #TODO 1: Add your code here
    df_eng = df_eng.rename(columns={"Movie / TV Series": "Title"})
    df_eng.insert(4, "Original Language", ["en" for _ in range(len(df_eng))], True)
    
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    df_sp = df_sp.rename(columns={"Título": "Title", "Año": "Year", "Sinopsis": "Synopsis", "Críticas": "Review"})
    df_sp.insert(4, "Original Language", ["sp" for _ in range(len(df_sp))], True)

    
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_fr = df_fr.rename(columns={"Titre": "Title", "Année": "Year", "Synopsis": "Synopsis", "Critiques": "Review"})
    df_fr.insert(4, "Original Language", ["fr" for _ in range(len(df_fr))], True)

    df = pd.concat([df_eng, df_sp, df_fr], ignore_index=True)
    
    return df

df = preprocess_data()

In [3]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",en
14,Amores perros,2000,Tres historias se entrelazan en esta película ...,"""Amores perros es una película intensa y conmo...",sp
23,Les Choristes,2004,Ce film raconte l'histoire d'un professeur de ...,"""Les Choristes est un film magnifique qui vous...",fr
16,Toc Toc,2017,"En esta comedia española, un grupo de personas...","""Toc Toc es una película aburrida y poco origi...",sp
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",en
17,El Bar,2017,Un grupo de personas quedan atrapadas en un ba...,"""El Bar es una película ridícula y sin sentido...",sp
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",en
24,Le Fabuleux Destin d'Amélie Poulain,2001,Cette comédie romantique raconte l'histoire d'...,"""Le Fabuleux Destin d'Amélie Poulain est un fi...",fr
26,La Tour Montparnasse Infernale,2001,Deux employés de bureau incompétents se retrou...,"""Je ne peux pas croire que j'ai perdu du temps...",fr
27,Astérix aux Jeux Olympiques,2008,Dans cette adaptation cinématographique de la ...,"""Ce film est une déception totale. Les blagues...",fr


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [4]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt")

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

In [5]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews = df.loc[df["Original Language"] == "fr"]["Review"]
fr_reviews_en = [translate(review, fr_en_model, fr_en_tokenizer) for review in fr_reviews]
# print(fr_reviews_en)

# filter synopsis in French and translate to English
fr_synopsis = df.loc[df["Original Language"] == "fr"]["Synopsis"]
fr_synopsis_en = [translate(syn, fr_en_model, fr_en_tokenizer) for syn in fr_synopsis]

# filter reviews in Spanish and translate to English
es_reviews = df.loc[df["Original Language"] == "sp"]["Review"]
es_reviews_en = [translate(review, es_en_model, es_en_tokenizer) for review in es_reviews]

# filter synopsis in Spanish and translate to English
es_synopsis = df.loc[df["Original Language"] == "sp"]["Synopsis"]
es_synopsis_en = [translate(syn, es_en_model, es_en_tokenizer) for syn in es_synopsis]

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data
df.loc[df["Original Language"] == "fr", "Review"] = fr_reviews_en
df.loc[df["Original Language"] == "fr", "Synopsis"] = fr_synopsis_en
df.loc[df["Original Language"] == "sp", "Review"] = es_reviews_en
df.loc[df["Original Language"] == "sp", "Synopsis"] = es_synopsis_en

/home/mwalden/miniconda3/envs/udacity/lib/python3.7/site-packages/transformers/generation/utils.py:1357: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [6]:
df.sample(30)

,Title,Year,Synopsis,Review,Original Language
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",en
13,El Laberinto del Fauno,2006,"During the Spanish postwar period, Ofelia (Iva...","""The Labyrinth of Fauno is a fascinating and e...",sp
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",en
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",en
17,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ...",sp
28,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",fr
16,Toc Toc,2017,"In this Spanish comedy, a group of people with...","""Toc Toc is a boring and unoriginal film that ...",sp
12,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",sp
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",en
11,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ...",sp


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [7]:
# TODO 5: Update the code below
# load sentiment analysis model
from transformers import pipeline

model_name = "siebert/sentiment-roberta-large-english" 
sentiment_classifier = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")
# print(sentiment_classifier("Kiss my butt!")[0]["label"])

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    sentiment = classifier(text)[0]["label"]
    
    return sentiment

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [8]:
df.insert(4, "Sentiment", ["" for _ in range(len(df))], True)

In [9]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
for i in range(len(df)):
    df.loc[i, "Sentiment"] = analyze_sentiment(df.loc[i, "Review"], sentiment_classifier)

In [10]:
df.sample(10)

,Title,Year,Synopsis,Review,Sentiment,Original Language
22,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",POSITIVE,fr
21,Intouchables,2011,This film tells the story of the unlikely frie...,"""Untouchables is an incredibly touching film w...",POSITIVE,fr
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",NEGATIVE,en
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",POSITIVE,en
13,El Laberinto del Fauno,2006,"During the Spanish postwar period, Ofelia (Iva...","""The Labyrinth of Fauno is a fascinating and e...",POSITIVE,sp
19,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",NEGATIVE,sp
14,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",POSITIVE,sp
17,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ...",NEGATIVE,sp
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",POSITIVE,en
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",POSITIVE,en


In [11]:
# export the results to a .csv file
df.to_csv("reviews_with_sentiment.csv", index=False)